In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings('ignore')


import os
os.environ['KAGGLE_CONFIG_DIR']='/content'
# Loading dataset games.csv
df_games= pd.read_csv('../input/nfl-big-data-bowl-2022/games.csv')
#df_games = pd.read_csv('games.csv')
df_games.info()

In [ ]:
df_plays= pd.read_csv('../input/nfl-big-data-bowl-2022/plays.csv').fillna(0)
#df_plays = pd.read_csv('plays.csv').fillna(0)
df_plays.info()

In [ ]:
plt.figure(figsize=(25,4))
plt.grid(True)
bp_pivot = df_plays.groupby('specialTeamsResult').possessionTeam.count().reset_index()
sns.barplot(bp_pivot.specialTeamsResult, bp_pivot.possessionTeam)
plt.title('Special Teams Result')
plt.xticks(rotation='vertical')
plt.show()       

In [ ]:
plt.figure(figsize=(25,5))
plt.grid(True)
bp_pivot = df_plays.groupby('possessionTeam').playResult.sum().reset_index()
sns.barplot(bp_pivot.possessionTeam, bp_pivot.playResult)
plt.title('Team in relation to Play Result')
plt.show()

In [ ]:
plt.figure(figsize=(25,5))
plt.grid(True)
bp_pivot = df_plays.groupby('possessionTeam').penaltyYards.sum().reset_index()
sns.barplot(bp_pivot.possessionTeam, bp_pivot.penaltyYards)
plt.title('Team in relation to the Penalty Yards')
plt.show()

In [ ]:
bp_pivot = df_plays.groupby('penaltyJerseyNumbers').count().reset_index()[1:]
plt.figure(figsize=(25,5))
plt.grid(True)
bp_pivot = bp_pivot.loc[bp_pivot['playResult'] >= 3]
sns.barplot(x=bp_pivot.penaltyJerseyNumbers, y=bp_pivot.playResult, data=bp_pivot )
plt.title('Players Penalty')
plt.xticks(rotation='vertical')
plt.show()    

In [ ]:
#df_players = pd.read_csv('players.csv')
df_players= pd.read_csv('../input/nfl-big-data-bowl-2022/players.csv')
df_players.info()

In [ ]:
#Engineering new features 

#Position on the field
Offense = ['C','G','P','T','TE','WR','QB','RB','FB','HB','OG']
Defense = ['S','SS','CB','LB','DE','DT','DB','NT','MLB','ILB','FS','ILB']
Special = ['K','LS','OLB','OT']
aa = []

df_players.insert(7,'Game_Position', 'Offense')

aa =(pd.get_dummies(df_players['Position']).columns)
df_players.set_index('Position')

for ii in range(len(aa)):
    
    if aa[ii] in Defense:
        
        df_players.loc[df_players.Position == aa[ii],'Game_Position']='Defense'  
         
    elif aa[ii] in Special:
        
        df_players.loc[df_players.Position == aa[ii],'Game_Position']='Special'
                
           
#convert all hight to in and later create bins to simplify records
aa = pd.get_dummies(df_players['height']).columns

for i in range(len(aa)):
    if len(aa[i]) == 3:
        h1 = int(aa[i][0]) * 12 + int(aa[i][2])
        
    elif len(aa[i]) == 4:
        h1 = int(aa[i][0]) * 12 + int(aa[i][2:4])
          
    
    df_players.loc[df_players.height == aa[i],'height'] = h1

    
#Grouping by hight
bins=[65,68,70,72,75,78,80,82]
bin_labels=['65-67','68-69','70-71','72-74','75-77','78-79','80-81']
name_Group='Height_Group'
name_Column='height'

def Bins_function():
    df_players[name_Group]=pd.cut( df_players[name_Column],bins,right=True,labels=bin_labels)
     
Bins_function()

#updating age of all payers
df_players.birthDate=pd.to_datetime(df_players['birthDate']).dt.year

df_players.insert(8,'Age',0)
year = datetime.date.today().year
aa = pd.get_dummies(df_players['birthDate']).columns

for i in range(len(aa)):
    year2 = int(aa[i])
    Y1 = year-year2
    df_players.loc[df_players.birthDate == aa[i],'Age'] = Y1
    
#Grouping by Age
bins=[20,25,30,35,40,45,50,55]
bin_labels=['20-24','25-29','30-34','35-39','40-44','45-49','50-54']
name_Group='Age_Group'
name_Column='Age'
Bins_function()

#setting parameters for the Length of the kick to 10
df_kick1 = df_plays.loc[df_plays.kickLength >= 10].copy()
df_kick1 = df_kick1[['kickerId','kickLength']]
df_kick1 = df_kick1.sort_values('kickerId',ascending = True)

df_players.insert(9,'kickLength',0)
aa = pd.get_dummies(df_kick1['kickerId']).columns
K1 = (df_kick1.groupby(['kickerId']).max())

for i in range(len(aa)):
    
    K1 = (df_kick1.loc[(df_kick1['kickerId']==aa[i])]).max()
    k1 = K1.kickLength
    
    df_players.loc[df_players.nflId == aa[i],'kickLength'] = k1

plt.figure(figsize=(25,5))
plt.grid(True)
sns.barplot(x=df_players.Height_Group, y=df_players.kickLength, data=df_players, hue=df_players.Game_Position )
plt.title('Kick Length in relation to Height')
plt.show() 


In [ ]:
plt.figure(figsize=(25,5))
plt.grid(True)
sns.barplot(x=df_players.Age_Group, y=df_players.kickLength, data=df_players, hue=df_players.Game_Position )
plt.title('Kick Length in relation to Age')
plt.show()  

In [ ]:
#df_tracking2020 = pd.read_csv('tracking2020.csv')
df_tracking2020= pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2020.csv')
df_tracking2021 = df_tracking2020.loc[df_tracking2020['time'] >= '2021-01-01 00:15:00'].copy()
df_tracking2021 = df_tracking2021.loc[df_tracking2021['displayName'] != 'football'].copy()
df_tracking2021.reset_index()
